In [17]:
import datetime
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import sys

sys.path.append(os.environ["SEOHASONG_LIBRARY"])
from dic_to import dic2obj

                                
class Report():
    
    def __init__(self, db_path):
        
        self.source_path = os.path.join(db_path, "source")
        with open(os.path.join(db_path, "reference.json")) as f:
            self.ref = dic2obj.convert(json.load(f))
        with open(os.path.join(db_path, "coin_list.json")) as f:
            coins = json.load(f)
        with open(os.path.join(db_path, "template.json")) as f:
            template = json.load(f)
        
        self.current_time = datetime.datetime.now().strftime("%Y-%m-%d")
        self.coins = sorted(list(
            set(sum([coin.split("-") for coin in coins], []))
            -{"BTRX", "USDT"}
        ))
        self.report = {coin: template.copy() for coin in self.coins}
        
        self.source = {}
        filepath = os.path.join(db_path, "source")
        self.prev_source = (pd.read_pickle(filepath)
                            if os.path.isfile(filepath)
                            else {})
    
    def set_text(self, url):
        response = (self.prev_source[url]
                    if url in self.prev_source
                    else requests.get(url))
        self.source.update({url: response})

    def get_text(self, url):
        return self.source[url].text
    
    def save_source(self):
        pd.to_pickle(self.source, self.source_path)
    
    def fill_numbers(self):
        
        coins, source, report = self.coins, self.source, self.report
        
        site = self.ref.coinmarketcap
        self.set_text(site.quick_search)
        quick_search = json.loads(self.get_text(site.quick_search))
        
        blacklist = ["batcoin"]
        self.quick_search = list(filter(
            lambda dic: dic["slug"] not in blacklist,
            quick_search
        ))
        
        self.name_map = {dic["symbol"]: dic["slug"]
                         for dic in self.quick_search}
        self.urls = {coin: site.currencies+self.name_map[coin]
                     for coin in coins}
        for coin in coins:
            self.set_text(self.urls[coin])
            
        self.save_source()        
        
        urls = self.urls
        for coin in coins:
            text = self.get_text(urls[coin])
            bs = BeautifulSoup(text, "lxml")
            selector = "span[data-currency-volume]"
            value = bs.select_one(selector)["data-usd"]
            report[coin]["Daily liquidity"] = float(value)
            selector = "span[data-currency-market-cap]"
            value = bs.select_one(selector)["data-usd"]
            report[coin]["Market cap"] = float(value)

In [58]:
report = Report("./db")
report.fill_numbers()

In [130]:
coins = list(report.report.keys())[14:]

for coin in coins:
    print("[{}]".format(coin))
    print("coin market cap: {}".format(report.urls[coin]))
    print("website: ")
    print()
    paper = report.report[coin]
    for key in paper:
        print("{}: {}".format(key, paper[key]))
    print()
    print()

[KMD]
coin market cap: https://coinmarketcap.com/currencies/komodo
website: 

Base algorithm: None
Daily liquidity: 10627800.0
Market cap: 401100962.756
Mining: None
Summary: None


[LRC]
coin market cap: https://coinmarketcap.com/currencies/loopring
website: 

Base algorithm: None
Daily liquidity: 22199300.0
Market cap: 425933152.354
Mining: None
Summary: None


[LSK]
coin market cap: https://coinmarketcap.com/currencies/lisk
website: 

Base algorithm: None
Daily liquidity: 25431800.0
Market cap: 1232217422.39
Mining: None
Summary: None


[LTC]
coin market cap: https://coinmarketcap.com/currencies/litecoin
website: 

Base algorithm: None
Daily liquidity: 413140000.0
Market cap: 8225012389.56
Mining: None
Summary: None


[MONA]
coin market cap: https://coinmarketcap.com/currencies/monacoin
website: 

Base algorithm: None
Daily liquidity: 6566170.0
Market cap: 236910044.272
Mining: None
Summary: None


[NEO]
coin market cap: https://coinmarketcap.com/currencies/neo
website: 

Base algor

In [101]:
report.report[coin]["Summary"] = """
"""
report.report[coin]["Base algorithm"] = "POW"
report.report[coin]["Mining"] = True

In [102]:
print(report.report[coin])

{'Base algorithm': 'POW', 'Daily liquidity': 2091.0, 'Market cap': 658500.010209, 'Mining': True, 'Summary': '\n'}


In [99]:
report.save_source()